In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [4]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [7]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [8]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [41. 24. 28. 45. 51.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [1 2 0 0 0]
fare                : [19.5  65.   52.   35.5  26.55]
class               : [b'Second' b'Second' b'First' b'First' b'First']
deck                : [b'unknown' b'unknown' b'A' b'unknown' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'n' b'y' b'y' b'y']


In [9]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [15. 28. 28. 28. 51.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [1 0 0 0 0]
fare                : [211.337   8.663  29.7     8.05    7.054]
class               : [b'First' b'Third' b'First' b'Third' b'Third']
deck                : [b'B' b'unknown' b'C' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [10]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [14. 38. 31. 28. 28.]
n_siblings_spouses  : [5 0 1 1 0]
class               : [b'Third' b'First' b'First' b'First' b'Third']
deck                : [b'unknown' b'B' b'B' b'unknown' b'unknown']
alone               : [b'n' b'y' b'n' b'n' b'y']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 18. 39. 26. 28.]
n_siblings_spouses  : [0. 1. 1. 2. 0.]
parch               : [0. 0. 1. 0. 0.]
fare                : [  7.55  108.9    83.158   8.663  12.35 ]


In [12]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [13]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [14]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[34.     0.     0.    13.   ]
 [28.     0.     0.     8.05 ]
 [28.     0.     0.     7.25 ]
 [28.     0.     0.     7.896]
 [33.     0.     0.     9.5  ]]

[1 0 0 0 0]


In [15]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'female']
age                 : [60. 40. 28. 22. 28.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [75.25 31.    0.    9.35  8.05]
class               : [b'First' b'First' b'Second' b'Third' b'Third']
deck                : [b'D' b'A' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


In [16]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [17]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [18]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'male']
class               : [b'Third' b'Second' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'B' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'n' b'y' b'y' b'y' b'n']
numeric             : [[25.     1.     0.     7.925]
 [25.     0.     0.    13.   ]
 [19.     0.     0.    26.   ]
 [45.     0.     0.    26.55 ]
 [28.     1.     0.    15.5  ]]


In [20]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [21]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [22]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [23]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [24]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fb8980f6048>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [25]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[25.   ,  1.   ,  0.   ,  7.925],
       [18.   ,  1.   ,  0.   , 17.8  ],
       [28.   ,  0.   ,  0.   ,  7.729],
       [28.   ,  0.   ,  0.   ,  7.05 ],
       [31.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [26]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.37 ,  0.395, -0.479, -0.485],
       [-0.93 ,  0.395, -0.479, -0.304],
       [-0.13 , -0.474, -0.479, -0.488],
       [-0.13 , -0.474, -0.479, -0.501],
       [ 0.109, -0.474, -0.479, -0.392]], dtype=float32)

In [27]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [28]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [29]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [30]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [31]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [32]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.37   0.395
 -0.479 -0.485  0.     1.   ]


In [33]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [34]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [35]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 1s 10ms/step - loss: 0.5039 - accuracy: 0.7911
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4167 - accuracy: 0.8182
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4059 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3888 - accuracy: 0.8262
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3810 - accuracy: 0.8309
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3772 - accuracy: 0.8341
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3635 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3639 - accuracy: 0.8373
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3513 - accuracy: 0.8533
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3495 - accuracy: 0.856

In [36]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

53/Unknown - 0s 8ms/step - loss: 0.4387 - accuracy: 0.8371

Test Loss 0.43871724043252336, Test Accuracy 0.8371211886405945


In [37]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 53.02%  | Actual outcome:  SURVIVED
Predicted survival: 22.04%  | Actual outcome:  SURVIVED
Predicted survival: 77.68%  | Actual outcome:  DIED
Predicted survival: 15.18%  | Actual outcome:  SURVIVED
Predicted survival: 22.73%  | Actual outcome:  DIED
